In [1]:
!pip install python-dotenv
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

### **練習 #1**

使用 Extractive QA model 以 `pipeline` 做 question-answering 任務：
- **給定文本**：the name of repo is bert-base-uncased
- **問題目標**：問模型 repo 的名稱
- **預期答案**：bert-base-uncased

In [2]:
# TODO
# practice 1 不需要特別指定模型，pipeline 預設載入 distilbert-base-cased-distilled-squad,
# 其為 Extractive QA 類摘要模型

from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [3]:
# 定义文章内容
context = """
the name of repo is bert-base-uncased
"""

# 定义问题
question = "問模型 repo 的名稱"

# 使用问答模型回答问题
result = qa_pipeline(question=question, context=context)

# 输出答案
print("文章名称:", result['answer'])

文章名称: the name of repo is bert-base-uncased


利用 Hugging Face 的 text-generation model 與 Sentence Transformers embedding model 實作 QA 檢索聊天機器人。</p>
基於提供的資料集，使用 Embedding Cosine Similarity 檢索參考資料，再透過 LLM 生成答案。</p>

1. Baseline
   - 將 demo 中的資料，替換成我們提供 or 自己的資料集
   - 能夠檢索相似資料
   - 基於檢索的資料進行回答
2. Advanced（Optional）
   - Embedding 怎麼儲存？每次都要重新計算嗎？
   - 該如何處理太久以前的歷史資料？
   - 利用 Gradio or Hugging Face Spaces 部署、分享 Chatbot

In [4]:
from pprint import pprint

from torch import cuda, bfloat16

from transformers import pipeline
from transformers import Conversation
conversation = Conversation() # 建立一個對話 Conversation 物件

from transformers import BitsAndBytesConfig, AutoConfig, AutoModelForCausalLM, AutoTokenizer
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device)

from sentence_transformers import SentenceTransformer


embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")

cuda:0


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

因為載入模型較大，使用 T4 GPU 時建議進行量化，以下程式為量化處理過程，</p>
在此先不贅述，有興趣的可以參考 Hugging face 官方文件～</p>

與前面範例不同的是，模型載入方法，我們透過 `AuToModelForCausalLM` 實例化模型，將其作為參數傳入 `pipeline`。

In [6]:
model_id = 'MediaTek-Research/Breeze-7B-32k-Instruct-v1_0'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model_config = AutoConfig.from_pretrained(
    model_id
)

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

hf_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/512M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [7]:
chatbot = pipeline(
    "text-generation",
    model=hf_model,
    tokenizer=tokenizer, # Tokenizer，要與模型匹配，主要提供 chat 模式時的特殊符號
    max_new_tokens=1024, # 模型最多可以生成多少字
    return_full_text=False # 控制 pipeline 只輸出 AI Message
)

Demo

In [8]:
import pandas as pd

In [9]:
df_dora = pd.read_csv('./chinesedora.csv')

In [10]:
# 建立ls_data列表
qa_data = []

# 將tool_name與intro欄位內容以換行符號接起來並放入ls_data列表
for index, row in df_dora.iterrows():
    combined_string = f"{row['tool_name']}是一個道具可以用來：{row['intro']}"
    qa_data.append(combined_string)

# 顯示結果
print(qa_data)

['指引天使／引路天使是一個道具可以用來：它可以告訴你最好的路，不至於在路上出意外。', '雲朵凝固瓦斯／凝雲噴霧是一個道具可以用來：使用這個瓦斯的話，雲會凝固，變得可以奔走在雲上。', '交換繩是一個道具可以用來：兩人同時握住繩索的話，心還是一樣，但身體替換。', '室內旅行機是一個道具可以用來：可以顯示出各種地點的立體影像，但室內面積並不會隨之更動。會讓自己的家有旅行的味道。', '落難神明／天神機械人是一個道具可以用來：是一個難看老機器人。不過，幫助他的話，可以滿足3個請求。', '夢風鈴是一個道具可以用來：使用這個風鈴，就可以將睡覺的人叫起來(他們是夢遊狀態)，並且任憑差遣。', 'ＮＳ徽章／南北極徽章是一個道具可以用來：只要將自己與另外一個人貼上寫有「Ｎ」或「Ｓ」的徽章，就會變的像磁鐵一樣，同極相斥，異極相吸。', '阿拉丁神燈是一個道具可以用來：擦拭油燈的話，冒煙的機器人就會出現，不管什麼命令都聽。', '間諜工具組合／間諜套裝是一個道具可以用來：能用監視器看間諜人偶的「麥克風」和「」偵察的影像。', '聲音糖果／聲音糖是一個道具可以用來：只要利用這個錄製聲音，再給別人吃下去以後， 就可以得到被錄製聲音的特徵。', '進化退化放射線槍／進化退化光線槍是一個道具可以用來：被這光照射到，動物就可以進化(或是退化)。', '夢境電視／顯夢電視是一個道具可以用來：能除去別人看的夢的電視。', '算了算了棒／算啦棒是一個道具可以用來：用這個棒子將正在生氣的人的嘴巴堵住，並說「算了，算了」，那人氣就消了。', '助興樂團／增加氣氛樂團是一個道具可以用來：這兩個道具都是人型玩偶，都會播放音樂，讓氣氛變好；不同的是，只要讓小型插秧姑娘唱歌，就可以快速地完成插秧。', '四次元口袋／四度空間百寶袋是一個道具可以用來：哆啦A夢的道具都是放在這裡面的。四次元空間是個沒有大小的空間，要多大有多大。', '美食桌巾／美食枱布是一個道具可以用來：它可以幫你做出好吃的料理。無論甚麼料理都可以變出來。', '邱比特之箭／愛神之箭是一個道具可以用來：被箭射中的人，會對射箭的人產生好感。', '感動麥克風／感動擴音器是一個道具可以用來：使用這個麥克風，發出的聲音就會讓人感動不已。', '桃太郎丸子／桃太郎飯糰是一個道具可以用來：讓動物吃下，他會對你言聽計從。', '樵夫之泉／樵夫水池是一個

In [ ]:
qa_data = ['指引天使／引路天使\n它可以告訴你最好的路，不至於在路上出意外。',
      '雲朵凝固瓦斯／凝雲噴霧\n使用這個瓦斯的話，雲會凝固，變得可以奔走在雲上。',
      '交換繩\n兩人同時握住繩索的話，心還是一樣，但身體替換。',
      '室內旅行機\n可以顯示出各種地點的立體影像，但室內面積並不會隨之更動。會讓自己的家有旅行的味道。',
      '落難神明／天神機械人\n是一個難看老機器人。不過，幫助他的話，可以滿足3個請求。',
      '夢風鈴\n使用這個風鈴，就可以將睡覺的人叫起來(他們是夢遊狀態)，並且任憑差遣。',
      'ＮＳ徽章／南北極徽章\n只要將自己與另外一個人貼上寫有「Ｎ」或「Ｓ」的徽章，就會變的像磁鐵一樣，同極相斥，異極相吸。',
      '阿拉丁神燈\n擦拭油燈的話，冒煙的機器人就會出現，不管什麼命令都聽。',
      '間諜工具組合／間諜套裝\n能用監視器看間諜人偶的「麥克風」和「」偵察的影像。',
      '聲音糖果／聲音糖\n只要利用這個錄製聲音，再給別人吃下去以後， 就可以得到被錄製聲音的特徵。',
      '進化退化放射線槍／進化退化光線槍\n被這光照射到，動物就可以進化(或是退化)。',
      '夢境電視／顯夢電視\n能除去別人看的夢的電視。',
      '算了算了棒／算啦棒\n用這個棒子將正在生氣的人的嘴巴堵住，並說「算了，算了」，那人氣就消了。',
      '助興樂團／增加氣氛樂團\n這兩個道具都是人型玩偶，都會播放音樂，讓氣氛變好；不同的是，只要讓小型插秧姑娘唱歌，就可以快速地完成插秧。',
      '四次元口袋／四度空間百寶袋\n哆啦A夢的道具都是放在這裡面的。四次元空間是個沒有大小的空間，要多大有多大。',
      '美食桌巾／美食枱布\n它可以幫你做出好吃的料理。無論甚麼料理都可以變出來。',
      '邱比特之箭／愛神之箭\n被箭射中的人，會對射箭的人產生好感。',
      '感動麥克風／感動擴音器\n使用這個麥克風，發出的聲音就會讓人感動不已。',
      '桃太郎丸子／桃太郎飯糰\n讓動物吃下，他會對你言聽計從。',
      '樵夫之泉／樵夫水池\n把東西丟到這個泉裡，女神就會出現。如果你說了實話，他就會給你一個更好、更新的東西。',
      '更衣照相機／換衫相機\n選擇合適的衣服，拍攝一下馬上就能穿在身上。',
      '穿透環／穿牆圈\n可以穿過任何障礙。',
      '如果電話亭\n「如果這個世界是……」只要這樣對著話筒說話，就可以創造出你所說出的世界。但是當取消時，世界將會變成「平行世界」，雙方沒有瓜葛。',
      '翻譯蒟蒻／翻譯糕\n吃了這個蒟蒻，任何人說的任何話，我們都可以聽懂、而且可以溝通。',
      '縮小燈／縮小電筒\n被光照射到，東西會變小。',
      '記憶麵包\n學生們應該會特別喜歡這道具吧！這也是為什麼它能排到第五名的重要原因！只要吃下這個吐司，就能非常容易的記住事前寫在吐司上的事物。',
      '時光布／時間布\n這個時光包巾位列第四，因為它的功用實在太大了，包了的東西既可以變新，也可以變舊。',
      '竹蜻蜓\n雖然很委屈的排到第 3 名，但是卻是最常被哆啦A夢使用的道具！在一開始便已登場。只要把它放置在身體任何部分，就可依自己意思在天空中翱翔。內藏超小型電池，可時速 80 公里連續飛行 8 小時。若是間斷飛行，還可撐更久！',
      '時光機\n長 2.2 公尺，寬 1.6 公尺，高 1.28 公尺，重 538 公斤。除了時間移動之外，它也可以用來空間與空間間的互相移動。其實前面這三個名次，也就是哆啦A夢最常使用的三個道具，不過是名次調換而已。',
      '任意門／隨意門\n最受眾人喜愛，光榮的第一名是由「任意門」獲得！只要心中想著想去的地點，電腦就會傳達並歪曲所在地與目的地間的空間而將兩地靠攏，跨過門就可以到達。但是不能去距離十光年以上的行星，或者是電腦地圖上沒有的地區。是很常用的道具，使用率位列第三，僅次於竹蜻蜓、時光機。']


In [11]:
from typing import List
import numpy as np

def get_answer(query: str, source: List[str]):
    most_related_sentence = None
    max_similarity = 0

    for sentence in source:
        sim = calculate_cosine_similarity(
        embedding_model.encode(query),
        embedding_model.encode(sentence)
        )

        if sim > max_similarity:
            most_related_sentence = sentence
            max_similarity = sim

    return most_related_sentence

def calculate_cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [15]:
user_query = input(">>>")
conversation = Conversation()

while user_query.lower() != "bye":
    print(f"user: {user_query}")
    # 尋找最相似的文件
    answer = get_answer(user_query, qa_data)
    print(f"answer: {answer}")
    llm_input = f"""請你基於以下資訊回答使用者的問題
    {answer}
    ===
    問題：{user_query}
    """
    conversation.add_user_input(llm_input)
    # 將 conversation.messages 丟給 chatbot
    chatbot_result = chatbot(conversation.messages)[0]['generated_text']
    print(f"AI: {chatbot_result}")
    conversation.append_response(chatbot_result)

    user_query = input(">>>")


>>>在路上找路可以用甚麼工具
user: 在路上找路可以用甚麼工具
answer: 指引天使／引路天使是一個道具可以用來：它可以告訴你最好的路，不至於在路上出意外。
AI: 在路上找路可以使用的工具有很多，以下是一些常用的工具：

1. 指南針 (Compass)：可以測定自己的位置和目標方向之間的角度，自
行判定路線。

2. 地圖 (Map)：是平面的地理資訊，可以看到各地的比例尺，道路，地理
特徵等，幫助自己了解目標位置和路線。現在有電子地圖的應用程式，
可以在行動電話上使用。

3. 路標 (Signpost)：在道路上，可以看到路標的文字或箭頭，指示道路的
方向和目的地。

4. 問路 (Ask for directions)：可以向當地人或知道路的人詢問路線。

5. 導航 (GPS)：利用衛星定位技術，可以快速準確地得出自己的位置和目
標的最短路線。現在有很多導航的應用程式，可以在行動電話上使用。

6. 引路天使 (Guide Angel)：這是一個道具，可以告訴你最好的路，不至於
在路上出意外。

7. 智慧型手機：現在的智慧型手機已經可以涵蓋了很多工具的功能，可以
直接使用導航、地圖等功能。

總之，在路上找路可以使用的工具很多，可以根據自己的需求和情況來選擇。
>>>肚子餓可以吃記憶吐司嗎
user: 肚子餓可以吃記憶吐司嗎
answer: 記憶麵包是一個道具可以用來：學生們應該會特別喜歡這道具吧！這也是為什麼它能排到第五名的重要原因！只要吃下這個吐司，就能非常容易的記住事前寫在吐司上的事物。
AI: 記憶吐司是一個具有特殊功能的道具，可以幫助人記住事前寫在吐司上的事物。它對學習和記憶有明顯的幫助。然而，記憶吐司的功能主要集中在學習和記憶方面，而不是在飢餓的情況下提供立即的飽足感。

如果你是為了解決飢餓感而吃記憶吐司，它可能不能滿足你的需求。在飢餓時，建議選擇可以立即飽足的食物，如水果、堅果、牛奶、麵包等。如果是要利用其記憶功能，則可在學習、記憶後，搭配一般食物使用，以兼顧營養和記憶需求。
>>>bye


In [17]:
user_query = input(">>>")
conversation = Conversation()
conversation.add_message({"role": "system", "content": "你是個工具使用的機器人，請依據user的問題，選出適合的工具，回答工具的用法"})

while user_query.lower() != "bye":
    print(f"user: {user_query}")
    # 尋找最相似的文件
    answer = get_answer(user_query, qa_data)
    print(f"answer: {answer}")
    llm_input = f"""請你基於以下資訊回答使用者的問題
    {answer}
    ===
    問題：{user_query}
    """
    conversation.add_user_input(llm_input)
    # 將 conversation.messages 丟給 chatbot
    chatbot_result = chatbot(conversation.messages)[0]['generated_text']
    print(f"AI: {chatbot_result}")
    conversation.append_response(chatbot_result)

    user_query = input(">>>")


>>>在路上找路可以用甚麼工具
user: 在路上找路可以用甚麼工具
answer: 指引天使／引路天使是一個道具可以用來：它可以告訴你最好的路，不至於在路上出意外。
AI: 在路上找路可以用「指引天使／引路天使」這個工具。它可以告訴你最好的路，以避免在路上出意外。
>>>bye
